In [1]:
# Step 1: Importing Libraries
# Import necessary libraries for data handling and visualization
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
# Step 2: Creating the Dataset
# Simulate user behavior data
data = {
    "user_id": [1, 2, 3, 4, 5],
    "browsing_time": [5, 10, 3, 8, 12],  # in minutes
    "products_viewed": [3, 8, 2, 6, 10],
    "purchases": [1, 2, 0, 1, 3]
}

In [3]:
# Convert to DataFrame
df = pd.DataFrame(data)
df

,user_id,browsing_time,products_viewed,purchases
0,1,5,3,1
1,2,10,8,2
2,3,3,2,0
3,4,8,6,1
4,5,12,10,3


In [4]:
df.to_csv("dataset.csv", index=False) #saving

In [5]:
!pip install streamlit



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [6]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [7]:
import openai

In [ ]:
openai.api_key=""

In [9]:
#Function to apply k-means clustering
def segment_users(data):
  features = ["browsing_time", "products_viewed", "purchases"]
  # Standardize the data for better clustering performance
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(data[features])
  # Perform clustering with K=2 (2 user segments)
  kmeans = KMeans(n_clusters=2, random_state=42)
  df['segment'] = kmeans.fit_predict(scaled_data)
  return data


In [10]:
#Function to generate personalised recommendations using open AI
def generate_recommendations(segment_id, segment_prompts):
  prompt=segment_prompts.get(segment_id, "write a general product recommendation")
  response=openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role":"system","content":"you are a product recommendation system"},
          {"role":"user","content":prompt}
        ],
      max_tokens=100,
      temperature=0.7
  )
  return response['choices'][0]['message']['content'].strip()

In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

#streamlit app
st.title("User Segmentation and Personalized Recommendations")

#step1: upload csv file
upload_file=st.file_uploader("upload a csv file", type=["csv"])

if upload_file is not None:
  df=pd.read_csv(upload_file)
  st.subheader("uploaded data")
  st.write(df)

  #Ensure required columns are present
  required_columns=["browsing_time", "products_viewed", "purchases"]

  if not required_columns.issubset(df.columns):
    st.error(f"Required columns:{','.join(required_columns)} are missing in the uploaded dataset.")
  else:
    # step2: Segment users
    segmented_data=segment_users(df)
    st.subheader("Segmented User Data")
    st.write(segmented_data)

    #step3: Set Open AI Key
    openai_api_key=st.text_input("Enter your OpenAI API key",type="pasword")
    if openai_api_key:
      #setp4 : Generate personalised Recommendations
      segment_prompts={
          0: "Write a concise, budget-friendly product recommendation for tech-savvy users",
          1: "write an engaging product recommendation highlighting premium feature for tech enthusiasts."
      }
      segmented_data['personalized_message']=segmented_data['segment'].apply(lambda x: generate_recommendations(x, segment_prompts))
      st.subheader("Personalized Recommendations")
      st.write(segmented_data[['user_id','segment', 'personalized_message']])

      #step5 Visualize Clusters
      st.subheader("User Segmentation Clusters")
      fig, ax=plt.subplots(figsize=(8,5))
      scatter=ax.scatter(segmented_data['browsing_time'], segmented_data['products_viewed'], c=segmented_data['segment'], cmap='viridis', s=100)
      ax.set_xlabel('Browsing Time (minutes)')
      ax.set_ylabel('Products Viewed')
      ax.set_title('User Segmentation Clusters')
      ax.legend(*scatter.legend_elements(), loc='upper right', title='Segment')
      ax.add_artist(plt.legend(loc='upper right', title='Segment'))
      st.pyplot(fig)


Writing app.py


In [12]:
#installing local tunnel
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 4s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [13]:
#running streamlit in the backgroud
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
#Exposing the port 8501
!npx localtunnel --port 8501

⠙your url is: https://rotten-lamps-listen.loca.lt
